In [4]:
!pip uninstall opengradient

Found existing installation: opengradient 0.1.0
Can't uninstall 'opengradient'. No files were found to uninstall.


In [2]:
!python setup.py sdist bdist_wheel
!pip install dist/opengradient-0.1.0.tar.gz

running sdist
running egg_info
writing opengradient.egg-info/PKG-INFO
writing dependency_links to opengradient.egg-info/dependency_links.txt
writing requirements to opengradient.egg-info/requires.txt
writing top-level names to opengradient.egg-info/top_level.txt
reading manifest file 'opengradient.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'opengradient.egg-info/SOURCES.txt'
running check
creating opengradient-0.1.0
creating opengradient-0.1.0/opengradient.egg-info
creating opengradient-0.1.0/src
creating opengradient-0.1.0/tests
copying files to opengradient-0.1.0...
copying LICENSE -> opengradient-0.1.0
copying README.md -> opengradient-0.1.0
copying setup.py -> opengradient-0.1.0
copying opengradient.egg-info/PKG-INFO -> opengradient-0.1.0/opengradient.egg-info
copying opengradient.egg-info/SOURCES.txt -> opengradient-0.1.0/opengradient.egg-info
copying opengradient.egg-info/dependency_links.txt -> opengradient-0.1.0/opengradient.egg-info
copying openg

In [3]:
!pip show opengradient

Name: opengradient
Version: 0.1.0
Summary: A Python SDK for OpenGradient inference services
Home-page: https://github.com/OpenGradient
Author: OpenGradient
Author-email: oliver@opengradient.ai
License: 
Location: /Users/oliver/Desktop/code/opengradient/sdk
Requires: onnx, requests
Required-by: 


In [9]:
import site
import os

for path in site.getsitepackages():
    opengradient_path = os.path.join(path, 'opengradient')
    if os.path.exists(opengradient_path):
        print(f"opengradient found in: {opengradient_path}")
        print("Contents:")
        print(os.listdir(opengradient_path))

opengradient found in: /opt/miniconda3/envs/vanna-storage/lib/python3.12/site-packages/opengradient
Contents:
['client.py', '__init__.py', 'types.py', '__pycache__']


In [4]:
!pip install -r requirements.txt

In [3]:
from opengradient import Client
from opengradient.types import ModelInput, InferenceMode
import json

client = Client(wallet_address="0xeB155EcC59fcc2eBdeF6dEa1711597A5dacEfB30", private_key="cd09980ef6e280afc3900d2d6801f9e9c5d858a5deaeeab74a65643f5ff1a4c1")
client.upload("network.onnx")

AttributeError: 'Client' object has no attribute 'storage_url'

In [1]:
from opengradient import Client
from opengradient.types import ModelInput, InferenceMode
import json
import web3

print("Step 1: Importing modules")
print(f"ModelInput attributes: {ModelInput.__dict__}")

print("\nStep 2: Loading input data from JSON file")
with open('input.json', 'r') as file:
    input_data = json.load(file)
print(f"Loaded input data: {input_data}")

print("\nStep 3: Initializing client")
client = Client(wallet_address="0xYourWalletAddressHere", private_key="YourPrivateKeyHere")
print("Client initialized")
print(f"Client attributes: {client.__dict__}")

print("\nStep 4: Preparing model input")
try:
    numbers = input_data['input_data'][0]
    strings = []  # Assuming no string inputs are required
    model_input = ModelInput(numbers=numbers, strings=strings)
    print(f"ModelInput created successfully: {model_input}")
except Exception as e:
    print(f"Error creating ModelInput: {str(e)}")
    raise

print("\nStep 5: Setting model ID and inference mode")
model_id = "Qma4p7LEZ5Bxgg5igC7VPk5Js8Asx9KgfDHjPcXFYrjAVc"
inference_mode = InferenceMode.VANILLA
print(f"Model ID: {model_id}")
print(f"Inference mode: {inference_mode}")

print("\nStep 6: Checking Web3 initialization")
if hasattr(client, '_initialize_web3'):
    print("Calling _initialize_web3 method")
    try:
        client._initialize_web3()
    except AttributeError:
        print("AttributeError occurred during _initialize_web3. Attempting to modify the method.")
        def _initialize_web3(self):
            from web3 import Web3
            self.w3 = Web3(Web3.HTTPProvider(self.rpc_url))
            print(f"Web3 connection established: {self.w3.is_connected()}")
        client._initialize_web3 = _initialize_web3.__get__(client)
        client._initialize_web3()
else:
    print("_initialize_web3 method not found")

print(f"Client w3 attribute after initialization: {getattr(client, 'w3', 'Not found')}")

print("\nStep 7: Calling infer method")
try:
    # Check if w3 is None before calling infer
    if client.w3 is None:
        print("Web3 connection is None. Attempting to reinitialize...")
        from web3 import Web3
        client.w3 = Web3(Web3.HTTPProvider(client.rpc_url))
        print(f"Web3 connection reinitialized: {client.w3.is_connected()}")
    
    result = client.infer(model_id, inference_mode, model_input)
    print(f"Inference result: {result}")
except Exception as e:
    print(f"An error occurred during inference: {str(e)}")
    print(f"Error type: {type(e)}")
    import traceback
    print("Traceback:")
    print(traceback.format_exc())

print("\nStep 8: Debugging client attributes after inference attempt")
print(f"Client attributes after inference: {client.__dict__}")
if hasattr(client, 'w3'):
    print(f"Web3 connection details: {client.w3}")
    print(f"Web3 connected: {client.w3.is_connected()}")
    print(f"Web3 provider: {client.w3.provider}")
else:
    print("Web3 connection (w3) not found in client attributes")

DEBUG:web3.providers.HTTPProvider:Making request HTTP. URI: http://18.218.115.248:8545, Method: web3_clientVersion
DEBUG:web3._utils.http_session_manager.HTTPSessionManager:Session cached: http://18.218.115.248:8545, <requests.sessions.Session object at 0x1267e7bf0>
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 18.218.115.248:8545
DEBUG:urllib3.connectionpool:http://18.218.115.248:8545 "POST / HTTP/11" 200 43
DEBUG:web3.providers.HTTPProvider:Getting response HTTP. URI: http://18.218.115.248:8545, Method: web3_clientVersion, Response: {'jsonrpc': '2.0', 'id': 0, 'result': '0.38.5'}


Private Key: a534e54acff4b86b61e690b64fe343b3e748fcd9a3628019c1162ac5d2a0109c
Ethereum Address: 0x0a513e468a2Eb7aae884705e5c598b0f47fC49E2
Step 1: Importing modules
ModelInput attributes: {'__module__': 'opengradient.types', '__annotations__': {'numbers': typing.List[opengradient.types.NumberTensor], 'strings': typing.List[opengradient.types.StringTensor]}, '__dict__': <attribute '__dict__' of 'ModelInput' objects>, '__weakref__': <attribute '__weakref__' of 'ModelInput' objects>, '__doc__': 'ModelInput(numbers: List[opengradient.types.NumberTensor], strings: List[opengradient.types.StringTensor])', '__dataclass_params__': _DataclassParams(init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=False,match_args=True,kw_only=False,slots=False,weakref_slot=False), '__dataclass_fields__': {'numbers': Field(name='numbers',type=typing.List[opengradient.types.NumberTensor],default=<dataclasses._MISSING_TYPE object at 0x107c50e00>,default_factory=<dataclasses._MISSING_TYPE object at 